# Load the Dataset

In [ ]:
import xarray as xr

folder = '/kaggle/input/eof-results/Full Year'

ds2 = xr.open_dataset(f'{folder}/2023-2024.EOF.ER.nc')
ds1 = xr.open_dataset(f'{folder}/2023-2024.EOF.KW.nc')

ds4 = xr.open_dataset(f'{folder}/2018-2019.EOF.ER.nc')
ds3 = xr.open_dataset(f'{folder}/2018-2019.EOF.KW.nc')

ds5 = xr.open_dataset(f'{folder}/2014-2015.EOF.KW.nc')
ds6 = xr.open_dataset(f'{folder}/2014-2015.EOF.ER.nc')

In [ ]:
waves = {
    '2023': {'kw':None, 'er':None},
    '2018': {'kw':None, 'er':None},
    '2014': {'kw':None, 'er':None}
}

sla = {
    '2023': {'kw':None, 'er':None},
    '2018': {'kw':None, 'er':None},
    '2014': {'kw':None, 'er':None}
}

In [ ]:
years = ['2023', '2018', '2014']

In [ ]:
for n, ds in enumerate([ds1, ds2, ds3, ds4, ds5, ds6]):
    if n % 2 == 0:
        i = n // 2
        waves[str(years[i])]['kw'] = ds
    else: 
        i = n // 2
        waves[str(years[i])]['er'] = ds

# Data Visualization

## Equatorial Waves Pattern

In [ ]:
# Individual Event

import matplotlib.pyplot as plt 
from matplotlib.colors import TwoSlopeNorm 
from scipy.ndimage import gaussian_filter
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import pandas as pd 
import numpy as np

# Assuming 'waves' is already defined elsewhere in your session
kw = waves['2023']['kw']
er = waves['2023']['er']
time = kw.time
lon  = kw.lon

fig, ax = plt.subplots(ncols=2, figsize=(8,6), dpi=300, sharey=True, constrained_layout=True)
titles = ['(a) Kelvin Waves ', '(b) Equatorial Rossby Waves']

# Colorbar configuration
factor = 100
limit = 0.05 * factor
norm = TwoSlopeNorm(vmin=-limit, vcenter=0, vmax=limit)
cbarticks = np.arange(-limit, limit+0.5, 0.5)

# Axes configuration
def lon_format(v, _):
    # v is longitude in degrees east (0-360)
    if v > 180:
        return f'{360 - v:.0f}°W'
    elif v < 180:
        return f'{v:.0f}°E'
    else:
        return '180°'

# Convert time to pandas DatetimeIndex
dt_index = pd.to_datetime(time)
# Find the first index of each month
month_start_locs = dt_index.to_series().groupby([dt_index.year, dt_index.month]).head(1).index
month_start_labels = [d.strftime('%b %Y') for d in month_start_locs]

ntime = len(time)

for i, data in enumerate([kw, er]):
    
    if i == 0:
        comp1, comp2 = data['comps'].sel(lat=slice(-2,2)).values[:2] 
        pc1, pc2     = data['scores'].values[:2]
    else:
        comp1, comp2 = data['comps'].sel(lat=slice(-5,5)).values[:2]
        pc1, pc2     = data['scores'].values[:2]

    ssh =  pc1[:, None, None] * comp1[None, :, :] + pc2[:, None, None] * comp2[None, :, :]
    hov = ssh.mean(axis=1) * 100
    
    c = ax[i].contourf(lon, time, hov, norm=norm, cmap='RdBu_r', levels=cbarticks, extend='both')
    if i == 0:
        ref = c
        
    ax[i].yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax[i].xaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax[i].xaxis.set_major_formatter(FuncFormatter(lon_format))
    ax[i].set_xticks(np.linspace(150, 270, 5))
    ax[i].tick_params(axis='both', labelsize=8)
    ax[i].set_title(titles[i], fontsize=10, fontweight='bold')
    ax[i].set_xlabel('Longitude', fontsize=8)

# Y-axis label on left plot
ystep = 3
ax[0].set_ylabel('Time', fontsize=8)
ax[0].set_yticks(month_start_locs[::ystep])
ax[0].set_yticklabels(month_start_labels[::ystep])


# Colorbar
cbar_ticks = cbarticks[::2]
cbar = fig.colorbar(ref, ax=ax, orientation='horizontal', aspect=30, pad=0.02, ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=8)
cbar.set_label('Sea Level Anomaly (cm)', fontsize=8)
cbar.set_ticklabels([f"{tick:.1f}" for tick in cbar_ticks])

plt.show()
fig.savefig('EW.HOV.2023.2S-2N.5S-5N.png', dpi=1000, bbox_inches='tight')

In [ ]:
# Multiple events

import matplotlib.pyplot as plt 
from matplotlib.colors import TwoSlopeNorm 
from scipy.ndimage import gaussian_filter
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import pandas as pd 
import numpy as np 

hovmoller = {
    '2023': {'kw':None, 'er':None},
    '2018': {'kw':None, 'er':None},
    '2014': {'kw':None, 'er':None}
}

# Axes configuration
# x-axis 
lon = waves['2023']['kw']['lon']
def lon_format(v, _):
    # v is longitude in degrees east (0-360)
    if v > 180:
        return f'{360 - v:.0f}°W'
    elif v < 180:
        return f'{v:.0f}°E'
    else:
        return '180°'
# y-axis
# Create dummy time
time = waves['2023']['kw']['time']
# Convert dummy_time to pandas DatetimeIndex for easier handling
dt_index = pd.to_datetime(time)
# Create formatted month labels, e.g., 'Jan(0)' for 2023, 'Jan(1)' for 2024
month_labels = [
    f"{d.strftime('%b')}({d.year - 2023})"
    for d in dt_index
]

ntime = 730

# Colorbar configuration
factor = 100        # scale the unit
# kw 
limit1 = 0.05 * factor
norm1 = TwoSlopeNorm(vmin=-limit1, vcenter=0, vmax=limit1)
cbarticks1 = np.arange(-limit1, limit1+0.5, 0.5)

# er 
limit2 = 0.05 * factor
norm2 = TwoSlopeNorm(vmin=-limit2, vcenter=0, vmax=limit2)
cbarticks2 = np.arange(-limit2, limit2+0.5, 0.5)

fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(9,8), dpi=300, sharey=True, sharex=True, constrained_layout=True)


titles = np.array([
    ['(a) 2023/24 El Niño: KW', '(d) 2023/24 El Niño: ER'],
    ['(b) 2018/19 El Niño: KW', '(e) 2018/19 El Niño: ER'],
    ['(c) 2014/15 El Niño: KW', '(f) 2014/15 El Niño: ER']
])


for n, (year, wave) in enumerate(waves.items()):

    # subset the data 
    kw = wave['kw']
    er = wave['er']

    # calculate the ssh pattern
    # kw 
    pc1, pc2 = kw['scores'].values[:2]  # shape: (time,)
    comp1, comp2 = kw['comps'].sel(lat=slice(-2,2)).values[:2]  # shape: (lat, lon)
    
    ssh_kw = pc1[:, None, None] * comp1[None, :, :] + pc2[:, None, None] * comp2[None, :, :]

    # er 
    pc1, pc2 = er['scores'].values[:2]
    comp1, comp2 = er['comps'].sel(lat=slice(-5,5)).values[:2]
    
    ssh_er = pc1[:, None, None] * comp1[None, :, :] + pc2[:, None, None] * comp2[None, :, :]
    
    cargs = dict(cmap='RdBu_r')
    # For plotting, select a latitude band or mean over latitude, e.g. mean over lat:
    ssh_kw_hov = ssh_kw.mean(axis=1) * 100  # shape: (time, lon)
    ssh_er_hov = ssh_er.mean(axis=1) * 100  # shape: (time, lon)

    if ssh_kw_hov.shape[0] == 730:
        t = time[:730]
    elif ssh_kw_hov.shape[0] == 731:
        t = time
    
    c1 = ax[0, n].contourf(lon, t, ssh_kw_hov, **cargs, norm=norm2, levels=cbarticks2, extend='both')
    c2 = ax[1, n].contourf(lon, t[:730], ssh_er_hov[:730], **cargs, norm=norm2, levels=cbarticks2, extend='both')

    # title 
    ax[0, n].set_title(titles[n, 0], fontweight='bold', fontsize=10)
    ax[1, n].set_title(titles[n, 1], fontweight='bold', fontsize=10)
    
    
    # colorbar configuration
    if n == 0:
        bar = c1

        cbar_ticks = cbarticks1[::2]
        cbar = fig.colorbar(bar, ax=ax, orientation='horizontal', pad=0.02, aspect=40, ticks=cbar_ticks)
        cbar.ax.tick_params(labelsize=8)
        cbar.set_ticklabels([f"{tick:.1f}" for tick in cbar_ticks])
        cbar.set_label('Sea Level Anomaly (cm)', fontsize=8)
        
    for i in range(2):
        # configure the y-axis 
        ystep = 120
        ax[i, n].set_yticks(dt_index[:ntime:ystep])
        ax[i, n].set_yticklabels(month_labels[:ntime:ystep], fontsize=8)
        ax[i, n].yaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax[i, n].xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax[i, n].xaxis.set_major_formatter(FuncFormatter(lon_format))
        ax[i, n].tick_params(axis='both', labelsize=8)
        ax[i, n].set_xlabel("Longitude", fontsize=8)
        ax[i, 0].set_ylabel("Time", fontsize=8)
    
    # x-axis
    #ax[1, n].set_xlabel('Longitude', fontsize=8)
    ax[1, n].set_xlim(150, 270)
    
    print(f'{n+1}/3 Completed')
     
plt.show()
fig.savefig('EW.HOV.2S-2N.5S-5N.png',  dpi=1000, bbox_inches='tight')

## EOF Pattern

In [ ]:
var = 'comps'
eofs = {
    '2023' : {'KW': ds1[var], 'ER':ds2[var]},
    '2018' : {'KW': ds3[var], 'ER':ds4[var]},
    '2014' : {'KW': ds5[var], 'ER':ds6[var]}
}

var2 = 'evr'
evr = {
    '2023' : {'KW': ds1[var2], 'ER':ds2[var2]},
    '2018' : {'KW': ds3[var2], 'ER':ds4[var2]},
    '2014' : {'KW': ds5[var2], 'ER':ds6[var2]}
}

### Leading Modes

In [ ]:
from matplotlib.colors import TwoSlopeNorm
import matplotlib.ticker as ticker
import numpy as np
# Leading Modes 

# Axes configuration
# Axes format
lat_format = lambda v, _: f'{v:.0f}°N' if v > 0 else (f'{abs(v):.0f}°S' if v < 0 else '0')
def lon_format(v, _):
    # v is longitude in degrees east (0-360)
    if v > 180:
        return f'{360 - v:.0f}°W'
    elif v < 180:
        return f'{v:.0f}°E'
    else:
        return '180°'

events = ['2023/24 El Niño: Leading EOF Modes', 
          '2018/19 El Niño: Leading EOF Modes', 
          '2014/15 El Niño: Leading EOF Modes']

limit = 0.5
norm = TwoSlopeNorm(vmin=-limit, vcenter=0, vmax=limit)

cbarticks = np.round(np.arange(-limit, limit+0.0625, 0.0625), 2)

cargs = dict(cmap='RdBu_r', norm=norm, levels=cbarticks, extend='both')

subtitles = ['(a) Kelvin Waves', '(b) Equatorial Rossby Waves']

for i, (year, eof) in enumerate(eofs.items()):

    print(year)
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 5), dpi=300, sharey=True, sharex=True, constrained_layout=True)
    for j, (type, comp) in enumerate(eof.items()):
        

        comp1 = comp[0] * 100
        comp2 = comp[1] * 100

        evr1  = evr[year][type][0].values 
        evr2  = evr[year][type][1].values 

        lons  = comp1.lon
        lats  = comp1.lat

        c1 = ax[0, j].contourf(lons, lats, comp1, **cargs)
        c2 = ax[1, j].contourf(lons, lats, comp2, **cargs)
        
        ax[0, 0].set_title(subtitles[0])
        ax[0, 1].set_title(subtitles[1])

        # patch
        at = AnchoredText(f"EOF-1: {evr1:.0f}%", 
                prop=dict(size=8), frameon=True, loc='upper left')
        at.patch.set_edgecolor("grey")
        ax[0, j].add_artist(at)

        at = AnchoredText(f"EOF-2: {evr2:.0f}%", 
                prop=dict(size=8), frameon=True, loc='upper left')
        at.patch.set_edgecolor("grey")
        ax[1, j].add_artist(at)


        ax[1, j].set_xlabel('Longitude')
        ax[1, j].xaxis.set_major_formatter(FuncFormatter(lon_format))
        ax[j, 0].set_ylabel('Latitude')
        ax[j, 0].yaxis.set_major_formatter(FuncFormatter(lat_format))
    cbar = fig.colorbar(c1, ax=ax, orientation='horizontal', pad=0.05, aspect=40, ticks=cbarticks[::4])
    cbar.set_label('EOF Amplitude (cm)')
    fig.suptitle(events[i], fontweight='bold', fontsize=14)
    fig.savefig(rf"{year}.Leading_EOF_Modes.png", dpi=500)
    plt.show()

### All of the EOF modes

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.colors import TwoSlopeNorm 
from scipy.ndimage import gaussian_filter
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import pandas as pd 
import numpy as np 

# Leading Modes 

# Axes configuration
# Axes format
lat_format = lambda v, _: f'{v:.0f}°N' if v > 0 else (f'{abs(v):.0f}°S' if v < 0 else '0')
def lon_format(v, _):
    # v is longitude in degrees east (0-360)
    if v > 180:
        return f'{360 - v:.0f}°W'
    elif v < 180:
        return f'{v:.0f}°E'
    else:
        return '180°'

titles  = ['2023/24 El Niño:', '2018/19 El Niño:', '2014/15 El Niño:']

limit = 0.5
norm = TwoSlopeNorm(vmin=-limit, vcenter=0, vmax=limit)

cbarticks = np.round(np.linspace(-0.5, 0.5, 17),2)

cargs = dict(cmap='RdBu_r', norm=norm, levels=cbarticks, extend='both')

subtitles = ['Kelvin Waves Modes', 'Equatorial Rossby Waves Modes']

for i, (year, eof) in enumerate(eofs.items()):
    print(year)
    for j, (type, comps) in enumerate(eof.items()):
        print(type)
        fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(10, 8), dpi=300, sharey=True, sharex=True, constrained_layout=True)
        ax = ax.flatten()
        for k, (comp, ax) in enumerate(zip(comps, ax)):

            lons = comp.lon
            lats = comp.lat

            variance = evr[year][type][k]        

            c = ax.contourf(lons, lats, comp*100, **cargs)

            # patch
            at = AnchoredText(f"EOF-{k+1}: {variance:.0f}%", 
                    prop=dict(size=8), frameon=True, loc='upper left')
            at.patch.set_edgecolor("grey")
            ax.add_artist(at)

            if k in [8, 9]:
                ax.set_xlabel('Longitude')
                ax.xaxis.set_major_formatter(FuncFormatter(lon_format))
            if k in [0, 2, 4, 6, 8]:
                ax.set_yticks([-10, -5, 0, 5, 10])
                ax.set_ylabel('Latitude')
                ax.yaxis.set_major_formatter(FuncFormatter(lat_format))
        
        cbar = fig.colorbar(c, ax=fig.axes,  orientation='horizontal', pad=0.03, aspect=40, ticks=cbarticks[::4])
        cbar.set_label('EOF Amplitude (cm)')

        fig.suptitle(f"{titles[i]} {subtitles[j]}", fontweight='bold', fontsize=14)
        fig.savefig(rf"{year}.{type}.EOF_Modes.png", dpi=500, bbox_inches='tight')
        plt.close()

## Explained Variance Ratio

In [ ]:
var = 'evr'
evr = {
    '2023' : {'KW': ds1[var], 'ER':ds2[var]},
    '2018' : {'KW': ds3[var], 'ER':ds4[var]},
    '2014' : {'KW': ds5[var], 'ER':ds6[var]}
}

In [ ]:

import matplotlib.pyplot as plt 
from matplotlib.ticker import FuncFormatter
import numpy as np 

x = np.arange(1, 11)
evr_format = lambda v, _: f'{v:.0f}%'

colors  = ['tab:blue', 'tab:orange']
titles  = ['(a) 2023/24 El Niño:', '(b) 2018/19 El Niño:', '(c) 2014/15 El Niño:']

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 3), dpi=300, sharey=True, constrained_layout=True)

for i, (year, data) in enumerate(evr.items()):

    for j, (type, variance) in enumerate(data.items()):
        sum = variance.values.sum()
        args = dict(marker="o", color=colors[j], ls='None')
        mask = (x != 1) & (x != 2)
        ax[i].plot(x[~mask], variance[~mask], **args, alpha=1, label=f"{type}: {sum:.0f}%")
        ax[i].plot(x[mask], variance[mask], **args, alpha=0.5)
    
    ax[i].legend()
    ax[i].yaxis.set_major_formatter(FuncFormatter(evr_format))
    ax[i].set_xlabel('EOF Mode')
    ax[i].set_xticks(x)
    ax[i].set_title(f'{titles[i]} EOF Modes')
    ax[i].grid(alpha=0.5)
ax[0].set_ylabel('Explained Variance')
plt.show()
fig.savefig(rf"EVR.png", dpi=500)

## Principal Components

In [ ]:
var = 'scores'
pc = {
    '2023' : {'KW': ds1[var], 'ER':ds2[var]},
    '2018' : {'KW': ds3[var], 'ER':ds4[var]},
    '2014' : {'KW': ds5[var], 'ER':ds6[var]}
}

In [ ]:
# Individual Event


from matplotlib.colors import TABLEAU_COLORS as tableau 
wave_names = ['Kelvin Waves', 'Equatorial Rossby Waves']
colors  = list(tableau.keys())
titles  = ['2023/24 El Niño:', '2018/19 El Niño:', '2014/15 El Niño:']


for i, (year, data) in enumerate(pc.items()):

    fig, ax = plt.subplots(figsize=(8, 5), nrows=2, ncols=1, dpi=300, sharey=True)
    
    print(year)
    for j, (type, scores) in enumerate(data.items()):
        pc1, pc2 = scores[:2]

        lags = np.arange(-30, 31) if type == 'KW' else np.arange(-180, 181)
        # Compute lag correlation
        correlations = []
        for lag in lags:
            if lag < 0:
                corr = np.corrcoef(pc1[:lag].values, pc2[-lag:].values)[0, 1]
            elif lag > 0:
                corr = np.corrcoef(pc1[lag:].values, pc2[:-lag].values)[0, 1]
            else:
                corr = np.corrcoef(pc1.values, pc2.values)[0, 1]
            correlations.append(corr)
            # Plot correlation vs lags
        ax[j].plot(lags, correlations, color=colors[0] if type == 'KW' else colors[1])
    
        ax[j].axhline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5, zorder=0)
        ax[j].axvline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5, zorder=0)
        ax[j].set_ylabel('Correlation')
    
        at = AnchoredText(wave_names[0] if type == 'KW' else wave_names[1], 
                        prop=dict(size=8), loc='upper left')
        at.patch.set_edgecolor("white")
        ax[j].add_artist(at)
        
    ax[1].set_xlabel('Lags (days)')
    ax[0].set_title(f'{titles[i]} Correlation of PC1 and PC2', fontweight='bold')
    plt.tight_layout()
    plt.show()
    fig.savefig(f"{year}.Correlation.png", dpi=500, bbox_inches='tight')

In [ ]:
# Multiple Events


from matplotlib.colors import TABLEAU_COLORS as tableau 
wave_names = ['Kelvin Waves', 'Equatorial Rossby Waves']
colors  = list(tableau.keys())
titles  = ['(a) 2023/24 El Niño', '(b) 2018/19 El Niño', '(c) 2014/15 El Niño']


fig, ax = plt.subplots(figsize=(12, 6), nrows=2, ncols=3, dpi=500, sharey=True, constrained_layout=True)

for i, (year, data) in enumerate(pc.items()):    
    ax[0, i].set_title(titles[i])
    print(year)
    for j, (type, scores) in enumerate(data.items()):
        pc1, pc2 = scores[:2]

        lags = np.arange(-30, 31) if type == 'KW' else np.arange(-180, 181)
        # Compute lag correlation
        correlations = []
        for lag in lags:
            if lag < 0:
                corr = np.corrcoef(pc1[:lag].values, pc2[-lag:].values)[0, 1]
            elif lag > 0:
                corr = np.corrcoef(pc1[lag:].values, pc2[:-lag].values)[0, 1]
            else:
                corr = np.corrcoef(pc1.values, pc2.values)[0, 1]
            correlations.append(corr)
            # Plot correlation vs lags
        print(f"{year}: {np.round(np.max(correlations),2)} | {np.round(np.min(correlations),2)}")
        ax[j, i].plot(lags, correlations, color=colors[0] if type == 'KW' else colors[1])
    
        ax[j, i].axhline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5, zorder=0)
        ax[j, i].axvline(0, color='black', linestyle='--', linewidth=0.8, alpha=0.5, zorder=0)
        
    
        at = AnchoredText(wave_names[0] if type == 'KW' else wave_names[1], 
                        prop=dict(size=10), loc='upper left')
        at.patch.set_edgecolor("white")
        ax[j, i].add_artist(at)
        ax[j, i].set_yticks(np.arange(-1, 1.2, 0.2))
        if i == 0:
            ax[j, i].set_ylabel('Correlation')
        if j == 1:
            ax[j, i].set_xlabel('Lags (days)')
    #ax[0].set_title(f'{titles[i]} Correlation of PC1 and PC2', fontweight='bold')
fig.suptitle("Lag Correlation between PC1 and PC2", fontweight='bold', fontsize=14)
plt.show()
fig.savefig(f"PC.Correlation.png", dpi=500, bbox_inches='tight')

## Equatorial Wave Index

In [ ]:
var = 'amp'
wave_index = {
    '2023' : {'KW': ds1[var], 'ER':ds2[var]},
    '2018' : {'KW': ds3[var], 'ER':ds4[var]},
    '2014' : {'KW': ds5[var], 'ER':ds6[var]}
}

In [ ]:
# Individual Event

from matplotlib.colors import TABLEAU_COLORS as tableau 
wave_names = ['KW', 'ER']
titles  = ['2023/24 El Niño:', '2018/19 El Niño:', '2014/15 El Niño:']

colors  = list(tableau.keys())



for i, (year, data) in enumerate(wave_index.items()):
    print(year)
    
    fig, ax = plt.subplots(figsize=(8, 3), dpi=500, constrained_layout=True)
    ax.set_title(f'{titles[i]} Equatorial Wave Index', fontweight='bold')
    for j, (type, amp) in enumerate(data.items()):
        time = amp['time']
        mean = np.round(np.mean(amp.values), 2)
        std  = np.round(np.std(amp.values),  2)
        ax.plot(time[:730], amp[:730], color=colors[j], label=f"{wave_names[j]}: {mean:.2f} ± {std}")

    ax.set_yticks(np.arange(0, 3.0, 0.5))
    ax.set_xlim(time[0], time[-1])
    ax.set_xlabel('Time')
    ax.set_ylabel('Wave Index')
    ax.legend(fontsize=8)
    plt.show()
    fig.savefig(f"{year}.Wave_Index.png", dpi=500, bbox_inches='tight')

In [ ]:
# Comparison of Wave Index
from matplotlib.colors import TABLEAU_COLORS as tableau 
titles = ['(a) Kelvin Waves', '(b) Equatorial Rossby Waves']
labels = ['2023/24 El Niño', '2018/19 El Niño', '2014/15 El Niño']

colors  = list(tableau.keys())

# X-axis configuration
time = wave_index['2023']['KW']['time']
# Convert dummy_time to pandas DatetimeIndex for easier handling
dt_index = pd.to_datetime(time)
# Create formatted month labels, e.g., 'Jan(0)' for 2023, 'Jan(1)' for 2024
month_labels = [
    f"{d.strftime('%b')}({d.year - 2023})"
    for d in dt_index
]
styles = {
    '2023' : {'alpha':1,   'color':'tab:blue',   'ls':'-',  'label':labels[0]},
    '2018' : {'alpha':0.5, 'color':'tab:orange', 'ls':'--', 'label':labels[1]},
    '2014' : {'alpha':0.5, 'color':'tab:green',  'ls':':',  'label':labels[2]},
}

fig, ax = plt.subplots(figsize=(8, 5), nrows=2, ncols=1, dpi=500, constrained_layout=True, sharex=True)

for i, (year, data) in enumerate(wave_index.items()):
    print(year)
    
    for j, (type, amp) in enumerate(data.items()):
        
        mean = np.round(np.mean(amp.values), 2)
        std  = np.round(np.std(amp.values),  2)
        ax[j].plot(time[:730], amp[:730], **styles[year])

    if i != 2:
        ax[i].set_title(titles[i], fontweight='bold')
        ax[i].set_xlim(time[0], time[-1])
        ax[i].set_ylabel('Wave Index')
        ax[i].tick_params(axis='both', labelsize=8)
        
    ax[0].set_yticks(np.arange(0, 3.5, 0.5))
    ax[1].set_yticks(np.arange(1.0, 1.7, 0.1))
    ax[1].set_xlabel('Time', fontsize=8)
    ax[1].set_xticks(dt_index[::3*30])
    ax[1].set_xticklabels(month_labels[::3*30])

leg = ax[1].legend(fontsize=8, loc='lower left', ncols=3, bbox_to_anchor=(0.2, -0.4))
plt.show()
fig.savefig("Wave_Index.png", dpi=500, bbox_inches='tight')

## Phase Speed of the Equatorial Waves

In [ ]:
import pandas as pd 

folder = "/kaggle/input/eof-results"

data2023 = pd.read_csv(fr'{folder}/2023-2024.EOF.Wave_Speeds.csv')
data2018 = pd.read_csv(fr'{folder}/2018-2019.EOF.Wave_Speeds.csv')
data2014 = pd.read_csv(fr'{folder}/2014-2015.EOF.Wave_Speeds.csv')

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

ylabel_format = lambda v, _: f'2014/15\nEl Niño' if np.round(v, 1) == 0.2 else (f'2018/19\nEl Niño' if v == 0.4 else (f'2023/24\nEl Niño' if v == 0.6 else ''))

# plot
fig, ax = plt.subplots(figsize=(6, 3), ncols=2, nrows=1, constrained_layout=True, dpi=300)

y_axes = [0.2, 0.4, 0.6]

colors = ['tab:blue', 'tab:orange', 'tab:green']

for n, ds in enumerate([data2014, data2018, data2023]):
    color = colors[abs(n-2)]
    for i, (key, data) in enumerate(ds.iterrows()):
        year = f'{data["Start"][:4]}-{data["Stop"][:4]}'
        v = data['Mean']
        std = data['STD']
        
        if i == 0:
            ax[1].scatter(v, y_axes[n], color=color)
            ax[1].set_xlim(2, 3)
            ax[1].text(v, y_axes[n]+0.02, f'{v:.2f}±{std:.2f}', fontsize=6, ha='center', va='bottom')
        else:
            ax[0].scatter(v, y_axes[n], color=color)
            ax[0].set_xlim(0, 1)
            ax[0].text(v, y_axes[n]+0.02, f'{v:.2f}±{std:.2f}', fontsize=6, ha='center', va='bottom')

        ax[i].xaxis.set_minor_locator(ticker.AutoMinorLocator())
        for label in ax[i].get_xticklabels():
            label.set_fontsize(8)
        ax[i].set_ylim(0.00, 0.80)
        
ax[0].set_yticks(y_axes)
ax[1].set_yticks([])
ax[0].yaxis.set_major_formatter(FuncFormatter(ylabel_format))
for label in ax[0].get_yticklabels():
    label.set_fontsize(8)

titles = ['(a) Equatorial Rossby Waves Speed', '(b) Kelvin Waves Speed']
for i in range(2):
    ax[i].set_title(titles[i], fontweight='bold', fontsize=8)

fig.supxlabel('Wave Speed (m/s)', fontsize=8)

fig.savefig(fr"Wave_Speeds.png", dpi=500)
plt.show()